In [15]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, Sequential, optimizers, losses, datasets

In [5]:
x = tf.constant([2., 1., 0.1])
layer = layers.Softmax(axis=-1)
np.sum(layer(x))

1.0000001

In [8]:
network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
])
x = tf.random.normal([4,3])
network(x)

<tf.Tensor: id=65, shape=(4, 2), dtype=float32, numpy=
array([[0.       , 2.1132584],
       [0.       , 5.2576118],
       [0.       , 3.0272255],
       [0.       , 0.       ]], dtype=float32)>

In [10]:
layers_num = 2
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())
network.build(input_shape=(None, 4))
network.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_6 (ReLU)               multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_7 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [24]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_8/kernel:0 (784, 256)
dense_8/bias:0 (256,)
dense_9/kernel:0 (256, 128)
dense_9/bias:0 (128,)
dense_10/kernel:0 (128, 64)
dense_10/bias:0 (64,)
dense_11/kernel:0 (64, 32)
dense_11/bias:0 (32,)
dense_12/kernel:0 (32, 10)
dense_12/bias:0 (10,)


In [26]:
def preprocess(x, y):
    """
    x is a simple image, not a batch
    """
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x,y


batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()

db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz) 

network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()


network.compile(optimizer=optimizers.Adam(lr=0.01),
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

history = network.fit(db, epochs=5, validation_data=ds_val, validation_freq=2)
network.save("model.h5")
#network = tf.keras.models.load_model('model.h5')

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             multiple                  200960    
_________________________________________________________________
dense_19 (Dense)             multiple                  32896     
_________________________________________________________________
dense_20 (Dense)             multiple                  8256      
_________________________________________________________________
dense_21 (Dense)             multiple                  2080      
_________________________________________________________________
dense_22 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
469/469 [==============================] - 14s 29ms/step - loss: 0.2695 - accuracy: 0.9176
Epoch 2/5
469

In [27]:
tf.keras.experimental.export_saved_model(network, 'model-savedmodel')
network = tf.keras.experimental.load_from_saved_model('model-savedmodel')

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             multiple                  200960    
_________________________________________________________________
dense_19 (Dense)             multiple                  32896     
_________________________________________________________________
dense_20 (Dense)             multiple                  8256      
_________________________________________________________________
dense_21 (Dense)             multiple                  2080      
_________________________________________________________________
dense_22 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
